In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import sys

In [3]:
sys.path.append("/home/lemma/Documents/MAC-POSTS/side_project/network_builder")

In [4]:
from models import *
import MNMAPI
from MNM_mcnb import *

In [13]:
from exp_config import *

Creating route for: single_drive
Creating route for: single_drive
Creating route for: multiple_drive
Creating route for: metro
Creating route for: pnr
Creating route for: pnr
Creating route for: pnr
Creating route for: pnr
Creating route for: single_drive
Creating route for: single_drive
Creating route for: multiple_drive
Creating route for: transit
Creating route for: pnr
Creating route for: single_drive
Creating route for: single_drive
Creating route for: multiple_drive
Creating route for: pnr


In [14]:
data_folder = '../data/input_files_small_multiclass'

In [15]:
nb = MNM_network_builder()

In [16]:
nb.load_from_folder(data_folder)

In [30]:
mcdue = Multimode_DUE(nb)

Init simulation


In [31]:
init_path_matrix = mcdue.init_path_matrix(path_list, demand_dict)

In [ ]:
mcdue.solve(init_path_matrix, path_list, demand_dict, ab_dict, 
                    choice_dict, num_iters = 100, gd_method = 'MSA', name = "trans5pnr5pool5")

1
2
11
12
13
14
15
16
17
18
